In [1]:
import pandas as pd
import json
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from textblob import TextBlob
import string

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_file = open("data/yelp_academic_dataset_review.json", encoding = "utf8")
reviews_data_0 = []
reviews_data_1 = []
reviews_data_2 = []
reviews_data_3 = []
reviews_data_4 = []
reviews_data_5 = []
reviews_data_6 = []
c = 0
for line in data_file:
    if c < 1000000:
        reviews_data_0.append(json.loads(line))
    elif c >= 1000000 and c < 2000000:
        reviews_data_1.append(json.loads(line))
    elif c >= 2000000 and c < 3000000:
        reviews_data_2.append(json.loads(line))
    elif c >= 3000000 and c < 4000000:
        reviews_data_3.append(json.loads(line))
    elif c >= 4000000 and c < 5000000:
        reviews_data_4.append(json.loads(line))
    elif c >= 5000000 and c < 6000000:
        reviews_data_5.append(json.loads(line))
    else:
        reviews_data_6.append(json.loads(line))
    c+=1

reviews_df_0 = pd.DataFrame(reviews_data_0)
reviews_df_1 = pd.DataFrame(reviews_data_1)
reviews_df_2 = pd.DataFrame(reviews_data_2)
reviews_df_3 = pd.DataFrame(reviews_data_3)
reviews_df_4 = pd.DataFrame(reviews_data_4)
reviews_df_5 = pd.DataFrame(reviews_data_5)
reviews_df_6 = pd.DataFrame(reviews_data_6)

data_file.close()

In [4]:
data_file = open("data/yelp_academic_dataset_business.json", encoding = "utf8")
business_data = []
for line in data_file:
    business_data.append(json.loads(line))

business_df = pd.DataFrame(business_data)

data_file.close()

In [ ]:
# reviews_df.sort_values('date')

In [5]:
# Filter out reviews for restaurants not in PA 
business_list = business_df[business_df['state']=='PA']['business_id']
# df = reviews_df_1[reviews_df_1['business_id'].isin(business_list)]

In [ ]:
# len(df)

In [6]:
def split_time_to_feature(df):

    df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
    
    df['year'] = pd.DatetimeIndex(df['date']).year
    df['month'] = pd.DatetimeIndex(df['date']).month
    df['day'] = pd.DatetimeIndex(df['date']).day
    df['hour'] = pd.DatetimeIndex(df['date']).hour
    df['min'] = pd.DatetimeIndex(df['date']).minute
    df['sec'] = pd.DatetimeIndex(df['date']).second

# split_time_to_feature(df)

In [ ]:
# Extract reivews left in 2000-2018
# df = df.loc[(df['year'] >= 2000) & (df['year'] <= 2018)]

### Feature engineering

In [7]:
def feature_engineering(df):
    # Creating a new column in the dataset for the length of the reviews
    df['length_of_reviews'] = df['text'].apply(len)

    # Creating a new column in the dataset for the number of words in the reviews
    df['num_of_words'] = df['text'].apply(lambda str:len(nltk.word_tokenize(str)))

    # Creating a new column in the dataset for the number of sentences in the reviews
    df['num_of_sentences'] = df['text'].apply(lambda paragraph:len(nltk.sent_tokenize(paragraph)))

    df['capital_words'] = df['text'].apply(lambda sen:len(re.findall(r'\b[A-Z]+\b', sen)))
    df['capital_words_ratio'] = df['capital_words']/df['num_of_words']
    df.drop(columns='capital_words', inplace = True)

### Dealing with texts in reviews

In [8]:
# Cleaning the reviews - stemed, remove stopwords and punctuation
def clean_text(text):

    ## Remove puncuation
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    ## Remove stop words
    nostop = [word for word in nopunc.split() if word.lower() not in stopwords.words('english') and len(word) >= 3]
    text = ' '.join(nostop)
   
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)    
    
    ps = PorterStemmer()
    text = ps.stem(text)

    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(text)

    return text

# df['cleaned_text'] = df['text'].apply(clean_text)

In [ ]:
# Sentiment analysis using textblob 
# df['sentiment_polarity'] = df['cleaned_text'].apply(lambda w:TextBlob(w).polarity)
# df['sentiment_subjectivity'] = df['cleaned_text'].apply(lambda w:TextBlob(w).subjectivity)

In [ ]:
reviews_list = [reviews_df_0, reviews_df_1, reviews_df_2, reviews_df_3,
               reviews_df_4, reviews_df_5, reviews_df_6]
for i, reviews in enumerate(reviews_list):
    df = reviews[reviews['business_id'].isin(business_list)]

    split_time_to_feature(df)

    # Extract reivews left in 2000-2018
    df = df.loc[(df['year'] >= 2000) & (df['year'] <= 2018)]

    feature_engineering(df)

    df['cleaned_text'] = df['text'].apply(clean_text)

    # Sentiment analysis using textblob 
    df['sentiment_polarity'] = df['cleaned_text'].apply(lambda w:TextBlob(w).polarity)
    df['sentiment_subjectivity'] = df['cleaned_text'].apply(lambda w:TextBlob(w).subjectivity)

    df.to_csv(f'data/cleaned_review_data_{i}.csv')